In [10]:
import numpy as np
from collections import defaultdict
from copy import deepcopy as copy
import networkx as nx

In [11]:
class Edge:
    def __init__(self, index:int =-1, symbol:str ='', degree:int=0, color: int=0,):
        self.index = index
        self.symbol = symbol
        self.color = color
        self.degree = degree

    def __gt__(self, other):
        try:
            return self.degree > other.degree
        except:
            return self.degree > other

    def __lt__(self, other):
        try:
            return self.degree < other.degree
        except:
            return self.degree < other

    def __eq__(self, other):
        try:
            return self.degree == other.degree
        except:
            return self.degree == other

class Vertex:
    def __init__(self, start, end, val=1):
        self.start = start
        self.end = end
        self.val = val

    def __gt__(self, other):
        try:
            return self.val > other.degree
        except:
            return self.val > other

    def __lt__(self, other):
        try:
            return self.val < other.degree
        except:
            return self.val < other

    def __eq__(self, other):
        try:
            return self.val == other.degree
        except:
            return self.val == other

In [12]:
class GraphMatrix:
    """
    通过传入结点symbol列表和以（起始节点symbol,终止节点symbol,权值）为元素的边列表来创建图
    """
    def __init__(self, edges=None, vertex=None, directed=False):
        self.directed = directed
        if edges is not None and vertex is not None:
            self.set(edges, vertex, directed)
        else:
            self.edges = []
            self.edgeNum = 0
            self.vertex = []
            self.vertexNum = 0
            self.matrix = [[]]
            self._symbol2index = None
        self.colorNum = 0
        self.coloredEdges = defaultdict(lambda : [])

    def __len__(self):
        return self.edgeNum

    def set(self, edges, vertex, directed=False):
        self.directed = directed

        self.edges = [Edge(index=i, symbol=sym) for i, sym in enumerate(edges)]
        self.edgeNum = len(edges)
        self._symbol2index = dict([(sym, i) for i, sym in enumerate(edges)])

        self.vertex = [Vertex(self._symbol2index[v[0]], self._symbol2index[v[1]], v[2]) for v in vertex]
        self.vertexNum = len(self.vertex)

        self.matrix = np.zeros([len(self.edges), len(self.edges)], dtype=int)
        self.matrix = [[] for _ in range(len(self.edges))]

        if self.directed:
            for v in self.vertex:
                self.matrix[v.start].append(v)
                self.edges[v.start].degree += 1
        if not self.directed:
            for v in self.vertex:
                self.matrix[v.end].append(v)
                self.edges[v.end].degree += 1

    def symbol2index(self, symbol):
        try:
            return self._symbol2index[symbol]
        except:
            return -1

    def index2symbol(self, index):
        if index < self.edgeNum:
            return self.edges[index].symbol
        else:
            return ''

    def isConnected(self, x, y):
        if x < self.edgeNum and y < self.edgeNum:
            if y in self.matrix[x]:
                return y.val
        else:
            return False

    def isColored(self, index):
        if isinstance(index, str):
            try:
                index = int(index)
            except:
                return False
        if isinstance(index, int) and index < len(self):
            return self.edges[index].color != 0
        else:
            return False

    def color(self):
        self.coloredEdges.clear()
        edges = sorted(self.edges, reverse=True)
        index = np.array([True for i in range(self.edgeNum)])
        self.colorNum = 0
        color = self.colorNum
        while len(edges) > 0:
            color += 1
            tar = edges[0]
            tar.color = color
            index[0] = False
            self.coloredEdges[color].append(tar.index)
            for e in self.matrix[tar.index]:
                self.edges[e.index].color = color
                index[e.index] = False
                self.coloredEdges[color].append(e.index)
            edges = edges[index]
            index = index[index]
        self.colorNum = color

    def showColor(self):
        for i in range(1, self.colorNum + 1):
            print('color',str(i),':')
            for e in self.coloredEdges[i]:
                print(self.index2symbol(e.index))

In [39]:
def getPathLen(graph, path):
    path_len = 0
    for i in path:
        path_len += graph._adj[i[0]][i[1]]['weight']
    return path_len

In [32]:
"""问题1"""
edges = 'a b c d e f g h i j k l m'.split()
vertex = [('a', 'd', 1), ('a', 'b', 4), ('a', 'c', 3), ('a', 'e', 1),
          ('b', 'd', 4), ('b', 'f', 3), ('b', 'c', 5),
          ('c', 'g', 1), ('c', 'm', 2),
          ('d', 'f', 2), ('d', 'h', 6),
          ('e', 'f', 4), ('e', 'k', 6), ('e', 'h', 3),
          ('f', 'g', 3), ('f', 'i', 4), ('f', 'l', 3),
          ('g', 'j', 4), ('g', 'm', 1),
          ('h', 'k', 3), ('h', 'l', 1),
          ('i', 'j', 6), ('i', 'l', 3),
          ('j', 'l', 4), ('j', 'm', 2),
          ('k', 'l', 3), ('k', 'i', 3),
          ('l', 'm', 5)]
graph = nx.Graph()
graph.add_nodes_from(edges)
graph.add_weighted_edges_from(vertex)

In [40]:
nx.spring_layout(graph)
print(nx.is_eulerian(graph))
euler = list(nx.eulerian_circuit(graph, ))
print(euler)
print('路径数', len(euler))
print('总长度', getPathLen(graph, euler))

True
[('a', 'e'), ('e', 'k'), ('k', 'l'), ('l', 'm'), ('m', 'j'), ('j', 'l'), ('l', 'i'), ('i', 'k'), ('k', 'h'), ('h', 'l'), ('l', 'f'), ('f', 'i'), ('i', 'j'), ('j', 'g'), ('g', 'm'), ('m', 'c'), ('c', 'g'), ('g', 'f'), ('f', 'e'), ('e', 'h'), ('h', 'd'), ('d', 'f'), ('f', 'b'), ('b', 'c'), ('c', 'a'), ('a', 'b'), ('b', 'd'), ('d', 'a')]
路径数 28
总长度 90


In [42]:
start = 'f'
euler = list(nx.eulerian_circuit(graph, start))
print('以指定节点', start, '为起始的路径为', euler)
print('路径数', len(euler))
print('总长度', getPathLen(graph, euler))

以指定节点 f 为起始的路径为 [('f', 'i'), ('i', 'k'), ('k', 'l'), ('l', 'm'), ('m', 'j'), ('j', 'l'), ('l', 'i'), ('i', 'j'), ('j', 'g'), ('g', 'm'), ('m', 'c'), ('c', 'g'), ('g', 'f'), ('f', 'l'), ('l', 'h'), ('h', 'k'), ('k', 'e'), ('e', 'h'), ('h', 'd'), ('d', 'f'), ('f', 'e'), ('e', 'a'), ('a', 'c'), ('c', 'b'), ('b', 'd'), ('d', 'a'), ('a', 'b'), ('b', 'f')]
路径数 28
总长度 90


In [6]:
class Team:
    def __init__(self, posNum, person):
        self.posNum = posNum
        self.personName = copy([p[0] for p in person])
        self.person = [[i, p[1]] for i,p in enumerate(person)]
        self.candidate = [[] for i in range(posNum)]
        for i in range(len(self.person)):
            self.checkin(self.candidate, i)
        self.final = []

    def gen(self):
        final = []
        self._gen(copy(self.candidate), final)
        return final

    def checkout(self, candidate, pIndex):
        person = self.person[pIndex]
        for p in person[1]:
            candidate[p].remove(pIndex)

    def checkin(self, candidate, pIndex):
        person = self.person[pIndex]
        for p in person[1]:
            candidate[p].append(pIndex)

    def _gen(self, candidate, final):
        for c in candidate[len(final):]:
            if len(c) == 0:
                return None
        if len(final) == len(candidate):
            self.final.append(copy(final))
        else:
            tmpC = copy(candidate[len(final)])
            for per in tmpC:
                final.append(per)
                self.checkout(candidate, per)
                self._gen(candidate, final)
                self.checkin(candidate, per)
                final.pop(-1)

    def show(self):
        for team in self.final:
            print([[i+1, self.personName[p]]for i,p in enumerate(team)])

In [7]:
personName = 'Allen Bob Chris Doug Eric Fred Gale Head'.split()
person = [[personName[0], [0, 1]], [personName[1], [0]], [personName[2], [0,1]], [personName[3], [2,3,4]], [personName[4], [1]], [personName[5], [0]], [personName[6], [2,3]], [personName[7], [1,2]]]
team = Team(5, person)
team.gen()
team.show()
print(len(team.final))

[[1, 'Allen'], [2, 'Chris'], [3, 'Head'], [4, 'Gale'], [5, 'Doug']]
[[1, 'Allen'], [2, 'Eric'], [3, 'Head'], [4, 'Gale'], [5, 'Doug']]
[[1, 'Bob'], [2, 'Chris'], [3, 'Head'], [4, 'Gale'], [5, 'Doug']]
[[1, 'Bob'], [2, 'Eric'], [3, 'Head'], [4, 'Gale'], [5, 'Doug']]
[[1, 'Bob'], [2, 'Allen'], [3, 'Head'], [4, 'Gale'], [5, 'Doug']]
[[1, 'Chris'], [2, 'Eric'], [3, 'Head'], [4, 'Gale'], [5, 'Doug']]
[[1, 'Chris'], [2, 'Allen'], [3, 'Head'], [4, 'Gale'], [5, 'Doug']]
[[1, 'Fred'], [2, 'Eric'], [3, 'Head'], [4, 'Gale'], [5, 'Doug']]
[[1, 'Fred'], [2, 'Allen'], [3, 'Head'], [4, 'Gale'], [5, 'Doug']]
[[1, 'Fred'], [2, 'Chris'], [3, 'Head'], [4, 'Gale'], [5, 'Doug']]
10


In [8]:
class Man:
    def __init__(self, index, pref, gen):
        self.pref = pref
        self.index = index
        self.couple = None
        self.cRank = np.inf
        self.gen = gen

def coupling(men, women):
    last_rank = len(men)
    for i in range(len(women)):
        for m in men:
            if m.couple is None:
                w = women[m.pref[i]-1]
                rank = w.pref.index(m.index+1)
                if rank < w.cRank:
                    if w.couple is not None:
                        w.couple.couple = None
                    w.couple = m
                    w.cRank = rank
                    m.couple = w

In [9]:
men = [[8,6,4,5,1,7,3,2],
       [8,2,3,1,6,5,4,7],
       [5,2,1,7,6,8,3,4],
       [7,6,3,8,5,4,1,2],
       [4,6,2,7,3,8,5,1],
       [4,5,8,6,3,7,1,2],
       [5,2,6,4,8,7,1,3],
       [6,1,4,3,8,7,2,5]]
men = [Man(i, men[i], 1) for i in range(len(men))]
women = [[4,3,8,1,5,2,6,7],
         [3,5,7,4,2,8,1,6],
         [8,1,3,7,2,4,5,6],
         [7,3,1,8,5,2,4,6],
         [3,7,1,8,4,5,6,2],
         [8,7,5,1,6,3,2,4],
         [4,7,8,3,2,6,5,1],
         [8,2,4,5,3,6,1,7]]
women = [Man(i, women[i], 0) for i in range(len(men))]
coupling(men, women)
for w in women:
    print('couple:', str(w.couple.index), str(w.index))

couple: 0 0
couple: 4 1
couple: 5 2
couple: 6 3
couple: 2 4
couple: 7 5
couple: 3 6
couple: 1 7
